In [1]:
from dictionaries import *
from helper_functions import *
from produce_datasets import *
import pandas as pd
import numpy as np

In [12]:
def add_qtrid(df):
    '''
    adds a column for the year and quarter.

    params: df(dataframe)
    returns: dataframe with column added'''
    df['qtrid'] = df['year'] + (df['qtr']/4)
    return df

def import_one(year):
    '''brings a single year's woth of data into a dataframe. Used for initial EDA. 
    Referenced in import_all

    params: year(str)
    returns: df(dataframe)'''
    filepath = '../data/area_files/' + str(year) + '.csv'
    #all relevant csvs are renamed with only the year
    df = pd.read_csv(filepath, dtype = schema_dict)
    #schema_dict is found in dictionaries.py
    for column in drop_columns:
        if column in df.columns:
            df = df.drop([column], axis = 1)
    return df

def import_all(years):
    '''combines as many years ofdata into a single dataframe, as well as adding quater id
    References import_one and add_qtrid

    params: years (list of str)
    returns: df (dataframe)'''
    df = import_one(years[0])
    for year in years[1:]:
        df = df.append(import_one(year))
    df = add_qtrid(df)
    return df

In [39]:
def create_timeline_2001(variable):
    '''produces a dataframe of the 2001 recession timeline.
    
    .0Used to compute targets

    params: variable, str, one of ['month3_emplvel' (employment), 'avg_wkly_wage' (wages)]
    returns: df, Dataframe
    exports a json file (used in plotting results)
    '''
    
    #create a dataframe of the years in question
    df = import_all(recession1_years)

    #drop 'unknown or undefined' areas
    df = df[~df['area_fips'].str.contains("999")]
    
    #pivots the table to arrange quarters in columns, drops extraneous variables.
    df = df.pivot_table(columns = 'qtrid', values = variable, index = ['area_fips', 'area_title'], aggfunc = np.sum)
    df = df.reset_index()
    
    #fill nans
    df = df.fillna(0)

    #creates a secondary dataframe with only timeline variables
    df2 = df.drop(columns = ['area_fips', 'area_title'])
    df2 = df2.reset_index()
    
    #drops the index so that all calculations are free of any type mismatches
    df2 = df2.drop(columns = 'index')
    df2 = df2.fillna(0)

### DEPRECIATED CODE #### 

#     #this specifies when the jobs numbers "bottom-out" during the recession
#     nadir = df2.iloc[:,6:].apply(lambda x: calc_nadir(x), axis=1).rename('nadir')
    
#     #counts the number of quarters to the nadir since the beginning of the timeframe
#     nadir_qtr = df2.iloc[:,6:].apply(lambda x: calc_nadir_qtr(x), axis=1).rename('nadir_qtr')
    
#     #computes the highest points before and after the nadir, and captures the quarter count
#     pre_peak = df2.apply(lambda x: calc_pre_peak(x), axis=1).rename('pre_peak')
#     pre_peak_qtr = df2.apply(lambda x: calc_pre_peak_quarter(x), axis=1).rename('pre_peak_qtr')
#     post_peak = df2.apply(lambda x: calc_post_peak(x), axis=1).rename('post_peak')
#     post_peak_qtr = df2.apply(lambda x: calc_post_peak_qtr(x), axis=1).rename('post_peak_qtr')

### DEPRECIATED CODE #### 
    
    #specifies the lowest job numbers during the recession. Disregards quarters before the recession event. 
    df2['nadir'] = df2.iloc[:,6:].min(axis=1)

    #specifies which quarter the nadir occured.
    df2['nadir_qtr'] = df2.iloc[:,6:].idxmin(axis=1).apply(lambda x: df.columns.get_loc(x))
    
    #creates a column to store indices for lookup.
    df2['new'] = [df2.iloc[i].values for i in df.index]
    
    #specifies the highest job numbers *before* the nadir.
    df2['pre_peak'] = df2.apply(lambda x: max(x['new'][0:x['nadir_qtr']]), axis=1)
    
    #specifies the highest job numbers *after* the nadir
    df2['post_peak'] = df2.apply(lambda x: max(x['new'][x['nadir_qtr']:]), axis=1)
    
    #specifies which quarter the pre-peak occurred.
    df2['pre_peak_qtr'] = pd.Series([s[i] for i, s in zip(df2.index, df2['pre_peak'].apply(
        lambda x: [i for i in (df2.iloc[:,0:-6] == x)
                   .idxmax(axis=1)]))]).apply(lambda x: df2.columns.get_loc(x))
    
    #specifies which quarter the post-peak occurred.
    df2['post_peak_qtr'] = pd.Series([s[i] for i, s in zip(df2.index, df2['post_peak'].apply(
        lambda x: [i for i in (df2.iloc[:,0:-6] == x)
                   .idxmax(axis=1)]))]).apply(lambda x: df2.columns.get_loc(x))
    
    #creates a new dataset to store derived fields
    df_new = df2[['nadir', 'nadir_qtr', 'pre_peak', 'pre_peak_qtr', 'post_peak', 'post_peak_qtr', 'new']]
    
    #puts the computed points in a dataframe, joins with timeline
    df = df.join(df_new, how = 'outer', rsuffix = '_derive')
    
    #PRIMARY TARGET: did the area decline the entire time(-1), did it start growing again but not avhieve it's former numbers(0), or did it grow and recover(1)?
    df['recovery'] = (df['post_peak'] >= df['pre_peak']) *1

    #SECONDARY TARGET: How long did the jobs numbers decline?
    df['decline'] = (df['nadir_qtr'] - df['pre_peak_qtr'])
    
    #TERTIARY TARGET: different in before/after jobs numbers
    df['delta'] = df['post_peak'] - df['pre_peak']
    
    #export the data
    # df.to_json('data/Recession1_timeline.json')
    return df

In [40]:
df = create_timeline_2001('month3_emplvl')
    

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4351 entries, 0 to 4350
Data columns (total 43 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   area_fips      4351 non-null   object 
 1   area_title     4351 non-null   object 
 2   2000.25        4351 non-null   float64
 3   2000.5         4351 non-null   float64
 4   2000.75        4351 non-null   float64
 5   2001.0         4351 non-null   float64
 6   2001.25        4351 non-null   float64
 7   2001.5         4351 non-null   float64
 8   2001.75        4351 non-null   float64
 9   2002.0         4351 non-null   float64
 10  2002.25        4351 non-null   float64
 11  2002.5         4351 non-null   float64
 12  2002.75        4351 non-null   float64
 13  2003.0         4351 non-null   float64
 14  2003.25        4351 non-null   float64
 15  2003.5         4351 non-null   float64
 16  2003.75        4351 non-null   float64
 17  2004.0         4351 non-null   float64
 18  2004.25 

In [41]:
df_samp = df[df['recovery'] == 1].sample(10)

In [49]:
df_samp['recovery_num'] = df_samp.apply(lambda x: (x['new'][x['nadir_qtr']:] > x['pre_peak']), axis=1)

In [54]:
df_samp['recovery_list_len'] = len(df_samp['new'])

In [56]:

df_samp['recovery_qtr'] = df_samp['recovery_num'].index(next(filter(lambda i: i !=0, df_samp['recovery_num'])))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
# biggest_vals = df_empl.transpose().iloc[0:8,:].max(axis=0)
# ending = (df_empl.transpose() > biggest_vals).idxmax()
# biggest_idx = df_wages.T.iloc[0:8,:].idxmax()